Author: Doug Branton?, Neven Caplar and the LINCC Frameworks team

Last updated: July 06, 2025

# Introduction to Nested-Pandas

This notebook explores the Nested-Pandas API, showing the basics of nesting data and touring the various ways of working with nested data.

In this notebook we'll show how to:
- Generate example data
- Understand nested-pandas dataframe
- Do filtering
- Run a function over nested dataframes
- Brin the result back and merge to our nested dataframe

## Exploring the NestedFrame Interface

In [1]:
import nested_pandas as npd
# import light_curve as licu
import pandas as pd
import numpy as np

from nested_pandas.utils import count_nested
from lsdb import ConeSearch

In [2]:
# Define the six fields from Data Preview 1 with RA and Dec coordinates
fields = {
    "ECDFS": (53.13, -28.10),  # Extended Chandra Deep Field South
    "EDFS": (59.10, -48.73),  # Euclid Deep Field South
    "Rubin_SV_38_7": (37.86, 6.98),  # Low Ecliptic Latitude Field
    "Rubin_SV_95_-25": (95.00, -25.00),  # Low Galactic Latitude Field
    "47_Tuc": (6.02, -72.08),  # 47 Tuc Globular Cluster
    "Fornax_dSph": (40.00, -34.45)  # Fornax Dwarf Spheroidal Galaxy
}

# Define a 2-degree (2*3600 arcseconds) search radius
radius_arcsec = 0.2 * 3600  # Convert 0.2 degree to arcseconds
# Create six cone searches
cones = {name: ConeSearch(ra=ra, dec=dec, radius_arcsec=radius_arcsec) for name, (ra, dec) in fields.items()}

In [3]:
# Load an example dataset
from nested_pandas.datasets import generate_data
ndf = generate_data(50,100, seed=1).rename({"nested": "lightcurve"}, axis=1)
ndf

a         b                                         lightcurve
0   0.417022  0.038734  [{t: 6.532898, flux: 77.388964, band: 'r'}; …]...
1   0.720324  1.357671  [{t: 10.541162, flux: 82.03493, band: 'r'}; …]...
..       ...       ...                                                ...
48  0.287775  0.005741  [{t: 11.19434, flux: 11.946375, band: 'r'}; …]...
49  0.130029  1.234290  [{t: 0.25112, flux: 94.006251, band: 'g'}; …] ...

[50 rows x 3 columns]

In [4]:
# Accessing an individual lightcurve dataframe
ndf["lightcurve"][0]

,t,flux,band
0,6.532898,77.388964,r
1,1.439486,55.886109,g
...,...,...,...
98,13.575110,76.549696,g
99,14.949946,14.803440,g


In [5]:
# flux is a sub-column of the "lightcurve" column
ndf["lightcurve.flux"]

0     77.388964
0     55.886109
        ...    
49    17.012447
49    86.404021
Name: flux, Length: 5000, dtype: double[pyarrow]

## Hands-on Scientific Example: Variability Analysis

### Load ZTF Timeseries Data

In [15]:
# only Load data around 
import lsdb

#ra_center, dec_center = cones["Rubin_SV_95_-25"].ra, cones["Rubin_SV_95_-25"].dec
# Load via LSDB, .compute() returns a NestedFrame to work with
# This may take a few minutes
ztf_ndf = lsdb.open_catalog('https://data.lsdb.io/hats/ztf_dr22/ztf_lc',
                            margin_cache='https://data.lsdb.io/hats/ztf_dr22/ztf_lc_10arcs',
                            search_filter=cones["Rubin_SV_95_-25"]).compute()
ztf_ndf.dtypes

objectid                    int64[pyarrow]
filterid                     int8[pyarrow]
fieldid                     int16[pyarrow]
rcid                         int8[pyarrow]
objra                       float[pyarrow]
objdec                      float[pyarrow]
nepochs                     int64[pyarrow]
hmjd        list<element: double>[pyarrow]
mag          list<element: float>[pyarrow]
magerr       list<element: float>[pyarrow]
clrcoeff     list<element: float>[pyarrow]
catflags     list<element: int32>[pyarrow]
Norder                      uint8[pyarrow]
Dir                        uint64[pyarrow]
Npix                       uint64[pyarrow]
dtype: object

In [18]:
# The timeseries data in this dataset is stored as lists, we can convert these into a nested column

ztf_ndf = ztf_ndf.nest_lists(columns=["hmjd","mag","magerr","clrcoeff","catflags"], name="timeseries")
ztf_ndf

objectid  filterid  fieldid  rcid      objra  \
_healpix_29                                                                
1450136709898271135  258108400000047         1      258    31  95.040352   
1450136709898294367  258208400000023         2      258    31  95.040352   
...                              ...       ...      ...   ...        ...   
1456155336767175581  258108100004227         1      258    28   94.99221   
1456155336769478700  258208100007410         2      258    28  94.992165   

                        objdec  nepochs  Norder  Dir  Npix  \
_healpix_29                                                  
1450136709898271135  -25.19565      157       3    0   321   
1450136709898294367 -25.195646      457       3    0   321   
...                        ...      ...     ...  ...   ...   
1456155336767175581 -24.804495       29       5    0  5173   
1456155336769478700 -24.804405      354       5    0  5173   

                                                            timeseries  
_healpix_29                                                             
1450136709898271135  [{hmjd: 58205.13356, mag: 16.904661, magerr: 0...  
1450136709898294367  [{hmjd: 58397.52197, mag: 16.151007, magerr: 0...  
...                                                                ...  
1456155336767175581  [{hmjd: 58423.449, mag: 19.983234, magerr: 0.1...  
1456155336769478700  [{hmjd: 58397.52198, mag: 19.669369, magerr: 0...  

[6984 rows x 11 columns]

### Perform Initial Filtering

In [19]:
# First filter given brightness of observations

ztf_ndf = ztf_ndf.query("timeseries.mag < 20.0")
ztf_ndf

objectid  filterid  fieldid  rcid      objra  \
_healpix_29                                                                
1450136709898271135  258108400000047         1      258    31  95.040352   
1450136709898294367  258208400000023         2      258    31  95.040352   
...                              ...       ...      ...   ...        ...   
1456155336767175581  258108100004227         1      258    28   94.99221   
1456155336769478700  258208100007410         2      258    28  94.992165   

                        objdec  nepochs  Norder  Dir  Npix  \
_healpix_29                                                  
1450136709898271135  -25.19565      157       3    0   321   
1450136709898294367 -25.195646      457       3    0   321   
...                        ...      ...     ...  ...   ...   
1456155336767175581 -24.804495       29       5    0  5173   
1456155336769478700 -24.804405      354       5    0  5173   

                                                            timeseries  
_healpix_29                                                             
1450136709898271135  [{hmjd: 58205.13356, mag: 16.904661, magerr: 0...  
1450136709898294367  [{hmjd: 58397.52197, mag: 16.151007, magerr: 0...  
...                                                                ...  
1456155336767175581  [{hmjd: 58423.449, mag: 19.983234, magerr: 0.1...  
1456155336769478700  [{hmjd: 58397.52198, mag: 19.669369, magerr: 0...  

[6984 rows x 11 columns]

In [22]:
# nepochs provides the unmodified number of observations, 
# but now that we've rejected some observations we'll need to recompute the counts

ztf_ndf = count_nested(ztf_ndf, "timeseries")
ztf_ndf

objectid  filterid  fieldid  rcid      objra  \
_healpix_29                                                                
1450136709898271135  258108400000047         1      258    31  95.040352   
1450136709898294367  258208400000023         2      258    31  95.040352   
...                              ...       ...      ...   ...        ...   
1456155336767175581  258108100004227         1      258    28   94.99221   
1456155336769478700  258208100007410         2      258    28  94.992165   

                        objdec  nepochs  Norder  Dir  Npix  \
_healpix_29                                                  
1450136709898271135  -25.19565      157       3    0   321   
1450136709898294367 -25.195646      457       3    0   321   
...                        ...      ...     ...  ...   ...   
1456155336767175581 -24.804495       29       5    0  5173   
1456155336769478700 -24.804405      354       5    0  5173   

                                                            timeseries  \
_healpix_29                                                              
1450136709898271135  [{hmjd: 58205.13356, mag: 16.904661, magerr: 0...   
1450136709898294367  [{hmjd: 58397.52197, mag: 16.151007, magerr: 0...   
...                                                                ...   
1456155336767175581  [{hmjd: 58423.449, mag: 19.983234, magerr: 0.1...   
1456155336769478700  [{hmjd: 58397.52198, mag: 19.669369, magerr: 0...   

                     n_timeseries  
_healpix_29                        
1450136709898271135           157  
1450136709898294367           457  
...                           ...  
1456155336767175581             2  
1456155336769478700           353  

[7308 rows x 12 columns]

In [23]:
# Now we can query by the newly added "n_timeseries" column
ztf_ndf = ztf_ndf.query("n_timeseries > 50")
ztf_ndf

objectid  filterid  fieldid  rcid      objra  \
_healpix_29                                                                
1450136709898271135  258108400000047         1      258    31  95.040352   
1450136709898294367  258208400000023         2      258    31  95.040352   
...                              ...       ...      ...   ...        ...   
1456155315393785536  258208100007398         2      258    28  94.959763   
1456155336769478700  258208100007410         2      258    28  94.992165   

                        objdec  nepochs  Norder  Dir  Npix  \
_healpix_29                                                  
1450136709898271135  -25.19565      157       3    0   321   
1450136709898294367 -25.195646      457       3    0   321   
...                        ...      ...     ...  ...   ...   
1456155315393785536 -24.803455      364       5    0  5173   
1456155336769478700 -24.804405      354       5    0  5173   

                                                            timeseries  \
_healpix_29                                                              
1450136709898271135  [{hmjd: 58205.13356, mag: 16.904661, magerr: 0...   
1450136709898294367  [{hmjd: 58397.52197, mag: 16.151007, magerr: 0...   
...                                                                ...   
1456155315393785536  [{hmjd: 58397.52198, mag: 19.500437, magerr: 0...   
1456155336769478700  [{hmjd: 58397.52198, mag: 19.669369, magerr: 0...   

                     n_timeseries  
_healpix_29                        
1450136709898271135           157  
1450136709898294367           457  
...                           ...  
1456155315393785536           364  
1456155336769478700           353  

[2897 rows x 12 columns]

### Calculate Periodograms for all Lightcurves

### Visualizing Results

### Using Results to Modify our NestedFrame

# Problem 2